In [1]:
import csv, sys

INPUT_ROWS_LIMIT= 100
FILENAME= 'dublinbikes_2020_Q1.csv'
MAX_STATION_ID= 117
MISSING_STATIONS= [117, 116, 70, 60, 46, 35, 20, 14, 1]

In [2]:
class StationConstantInfo: # this is a class to contain constant information about stations (so as to avoid unnecessary repeat reading of them)
    def __init__(self, station_id):
        self.station_id= station_id
        
    def populate(self, station_name, bike_capacity, address, latitude, longitude):
        self.station_name= station_name
        self.bike_capacity= bike_capacity
        self.address= address
        self.latitude= latitude
        self.longitude= longitude

In [3]:
station_id= []; date= []; time= []; available_bikes= [];
station_constants= [StationConstantInfo(i) for i in range(1, MAX_STATION_ID + 1)]
ids_to_populate= list(range(1, MAX_STATION_ID + 1))
for missing_id in MISSING_STATIONS:
    ids_to_populate.remove(missing_id)

with open(FILENAME, newline='') as f:
    reader = csv.reader(f); next(reader) # skip header
    current_id= 0
    try:
        while len(ids_to_populate) != 0:
            row= next(reader)
            if int(row[0]) == current_id: # this 'if' is just for performance
                continue
            current_id= int(row[0])
            if current_id in ids_to_populate:
                station_constants[current_id - 2].populate(row[3], row[4], row[8], row[9], row[10])
                ids_to_populate.remove(current_id)
        for missing_id in MISSING_STATIONS:
            station_constants.pop(missing_id - 2)
        reader = csv.reader(f); next(reader) # skip header
        for row_i, row in enumerate(reader):
            if row_i >= INPUT_ROWS_LIMIT:
                break
            station_id.append(row[0])
            date.append(row[1][0: 10])
            time.append(row[1][11: 19])
            available_bikes.append(row[5])
    except csv.Error as e:
        sys.exit('file {}, line {}: {}'.format(filename, reader.line_num, e))